## UNDERSTANDING DEEP LEARNING REQUIRES RE-THINKING GENERALIZATION

SOURCE: https://arxiv.org/pdf/1611.03530.pdf

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import time
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import pickle

In [11]:
input_channels = 3
num_classes = 10

device = 'cuda'
lr = 0.01
epochs = 32
batch_size = 80

### Dataset - CIFAR 10

In [12]:
img_transform = transforms.Compose([
    transforms.ToTensor(), # convert image to tensor
    transforms.CenterCrop(28) # crop from the center
])

def norm_image(data_sample):
    img_tensor = data_sample[0] # image tensors
    label = data_sample[1]

    img_means = img_tensor.mean(axis=[1,2]) # mean of image per channel
    img_sds = img_tensor.std(axis=[1,2]) # overall SD of image per channel

    mean_sub = img_tensor - img_means.unsqueeze(1).unsqueeze(2)
    img_norm = mean_sub.true_divide(img_sds.unsqueeze(1).unsqueeze(2))

    return (img_norm, label)

### Training and Testing Set

In [13]:
# training set

all_train = list(datasets.CIFAR10(root = 'data/', transform=img_transform, train = True, download=True))

random.shuffle(all_train)

train_data = all_train[:40000]
train_transformed = list(map(norm_image, train_data))
train_loader = DataLoader(dataset=train_transformed, batch_size=batch_size, shuffle=True)

val_data = all_train[40000:]
val_transformed = list(map(norm_image, val_data))
val_loader = DataLoader(dataset=val_transformed, batch_size=batch_size, shuffle=True)

test_data = datasets.CIFAR10(root='data/', transform=img_transform, train=False, download=True)
test_transformed = list(map(norm_image, val_data))
test_loader = DataLoader(dataset=val_transformed, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


## Implementation of Reduced GoogLeNet

In [14]:
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(conv_block, self).__init__()
        self.relu = nn.ReLU() # ReLU
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs) # convolution
        self.batchnorm = nn.BatchNorm2d(out_channels) # batch normalization

    def forward(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        return x

class inception_block(nn.Module):
    def __init__(self, in_channels, out_ch1, out_ch3):
        super(inception_block, self).__init__()
        # first conv module, same padding
        self.ch1 = conv_block(in_channels=in_channels, out_channels=out_ch1, kernel_size=(3,3), stride=(1,1), padding='same')
        # second conv module, same padding
        self.ch3 = conv_block(in_channels=in_channels, out_channels=out_ch3, kernel_size=(3,3), stride=(1,1), padding='same')

    def forward(self,x):
        # concatenate convolution outputs
        return torch.cat([self.ch1(x),self.ch3(x)],1)

class downsample_block(nn.Module):
    def __init__(self, in_channels, conv_out):
        super(downsample_block, self).__init__()
        # conv module
        self.convblock = conv_block(in_channels, conv_out, kernel_size=(3,3), stride=(2,2))
        # max pooling
        self.maxpool = nn.MaxPool2d(kernel_size=(3,3), stride=(2,2))

    def forward(self,x):
        return torch.cat([self.convblock(x),self.maxpool(x)],1)

class mini_GoogLeNet(nn.Module):
    def __init__(self,in_channels=3, num_classes=10, dropout_prob=0):
        super(mini_GoogLeNet, self).__init__()
        self.conv1 = conv_block(in_channels=3, out_channels=96, kernel_size=(3,3), stride=(1,1))
        self.inception1 = inception_block(96,32,32)
        self.inception2 = inception_block(64,32,48)

        self.downsample1 = downsample_block(80,80)
        self.inception3 = inception_block(160,112,48)
        self.inception4 = inception_block(160,96,64)
        self.inception5 = inception_block(160,80,80)
        self.inception6 = inception_block(160,48,96)

        self.downsample2 = downsample_block(144,96)
        self.inception7 = inception_block(240,176,160)
        self.inception8 = inception_block(336,176,160)
        self.avgpool = nn.AvgPool2d(kernel_size=(7,7), padding=(1,1))
        self.dropout = nn.Dropout(p = dropout_prob)
        self.fc = nn.Linear(336,10)

    def forward(self,x):
        x = self.conv1(x)
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.downsample1(x)
        x = self.inception3(x)
        x = self.inception4(x)
        x = self.inception5(x)
        x = self.inception6(x)
        x = self.downsample2(x)
        x = self.inception7(x)
        x = self.inception8(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0],-1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

## Model Training

In [15]:
model = mini_GoogLeNet(in_channels=3, num_classes=10, dropout_prob=0).to(device=device)

# parameter settings
loss_function = nn.CrossEntropyLoss() # loss funtion
optimizer = optim.SGD(model.parameters(), lr=lr) # optimizer
scheduler = optim.lr_scheduler.LinearLR(optimizer) # scheduler

In [16]:
# initialize results
train_acc = []
train_all_loss = []

test_acc = []
test_all_loss = []

con_mats = []

times = []
train_times = []

In [17]:
def train(epoch):
    model.train()
    curr_loss_train = 0 # loss
    correct_train = 0 # correctly classified training points
    total_train = 0 # total number of training points

    for ind, (data_train, true_labels_train) in enumerate(train_loader):
        data_train = data_train.to(device=device) # use GPU
        true_labels_train = true_labels_train.to(device=device) # use GPU

        out_train = model(data_train) # apply model to training data
        loss_train = loss_function(out_train, true_labels_train) # get loss

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        curr_loss_train += loss_train.item()
        ix, predicted_train = out_train.max(1)
        correct_train += predicted_train.eq(true_labels_train).sum().item()
        total_train += true_labels_train.size(0)

    train_loss = curr_loss_train/len(train_loader) # get loss
    acc_train_val = (correct_train/total_train)*100 # get accuracy

    train_acc.append(acc_train_val)
    train_all_loss.append(train_loss)

def test(epoch):
    model.eval()
    curr_loss_test = 0 # loss
    correct_test = 0 # correctly classified test points
    total_test = 0 # total number of test points
    num_class = 10
    confusion_matrix = torch.zeros(num_class, num_class)
    with torch.no_grad():
        for data_test, true_labels_test in test_loader:

            data_test = data_test.to(device=device) # use GPU
            true_labels_test = true_labels_test.to(device=device) # use GPU

            out_test = model(data_test) # apply model to training data
            loss_test = loss_function(out_test, true_labels_test) # get loss

            # metrics
            curr_loss_test += loss_test.item()
            ix, predicted_test = out_test.max(1)
            correct_test += predicted_test.eq(true_labels_test).sum().item()
            total_test += true_labels_test.size(0)


    test_loss = curr_loss_test/len(test_loader) # get loss
    acc_test_val = (correct_test/total_test)*100 # get accuracy

    test_acc.append(acc_test_val)
    test_all_loss.append(test_loss)
    con_mats.append(confusion_matrix)

### Metrics

In [18]:
train_start = time.time()
for epoch in range(epochs):
    ep_start = time.time()
    print(f"Epoch {epoch}")
    train(epoch)
    test(epoch)
    scheduler.step()

    epoch_time = time.time() - ep_start
    print(f"Epoch time: {epoch_time:0.2f} seconds")
    times.append(epoch_time)

    train_time = time.time() - train_start
    train_times.append(train_time)

print()
print(f"Total training time: {train_time:0.2f} seconds")

Epoch 0


Epoch time: 276.80 seconds
Epoch 1
Epoch time: 276.68 seconds
Epoch 2
Epoch time: 276.85 seconds
Epoch 3
Epoch time: 277.27 seconds
Epoch 4
Epoch time: 276.66 seconds
Epoch 5
Epoch time: 276.25 seconds
Epoch 6
Epoch time: 276.35 seconds
Epoch 7
Epoch time: 279.91 seconds
Epoch 8
Epoch time: 276.48 seconds
Epoch 9
Epoch time: 276.45 seconds
Epoch 10
Epoch time: 276.61 seconds
Epoch 11
Epoch time: 276.43 seconds
Epoch 12
Epoch time: 276.40 seconds
Epoch 13
Epoch time: 276.38 seconds
Epoch 14
Epoch time: 276.46 seconds
Epoch 15
Epoch time: 276.45 seconds
Epoch 16
Epoch time: 276.48 seconds
Epoch 17
Epoch time: 276.43 seconds
Epoch 18
Epoch time: 276.45 seconds
Epoch 19
Epoch time: 276.43 seconds
Epoch 20
Epoch time: 276.48 seconds
Epoch 21
Epoch time: 276.43 seconds
Epoch 22
Epoch time: 276.46 seconds
Epoch 23
Epoch time: 276.44 seconds
Epoch 24
Epoch time: 276.44 seconds
Epoch 25
Epoch time: 276.44 seconds
Epoch 26
Epoch time: 276.48 seconds
Epoch 27
Epoch time: 276.42 seconds
Epoch 28
E

## Export results

In [19]:
df_res = pd.DataFrame()
df_res['TrainAccuracy'] = train_acc
df_res['TrainLoss'] = train_all_loss
df_res['TestAccuracy'] = test_acc
df_res['TestLoss'] = test_all_loss
df_res['EpochTime'] = times
df_res['Totaltime'] = train_times

df_res.to_csv('19-results.csv', index=False)

In [20]:
with open('19-confusion_matrices.pickle','wb') as handle:
    pickle.dump(con_mats, handle)

## Results

In [21]:
pd.set_option('display.max_rows', None)
df_res

,TrainAccuracy,TrainLoss,TestAccuracy,TestLoss,EpochTime,Totaltime
0,31.0875,2.033667,39.02,1.767887,276.802399,276.803397
1,47.4500,1.555674,51.91,1.415237,276.683578,553.486975
2,57.6700,1.248891,53.47,1.345573,276.845322,830.332296
3,64.2925,1.049838,63.83,1.042297,277.266675,1107.599968
4,69.4450,0.897086,60.72,1.169229,276.662290,1384.262258
5,73.6975,0.779820,59.11,1.245290,276.246786,1660.509044
6,78.0550,0.660093,69.55,0.866342,276.353543,1936.862586
7,81.5425,0.558322,72.21,0.828857,279.914706,2216.777292
8,84.3575,0.480229,61.96,1.205901,276.479648,2493.256940
9,87.3425,0.397234,67.57,0.999281,276.447989,2769.704929


### Best Model Accuracy

In [22]:
max_accuracy = df_res['TestAccuracy'].max()
print(f"Max Accuracy: {max_accuracy}%")

Max Accuracy: 81.58%
